In [1]:
nb_users_per_sector  = 8;
nb_sectors           = 9;
nb_RBs               = 15;
total_nb_users       = nb_users_per_sector * nb_sectors;
min_power_per_RB     = 0.1;
max_power_per_sector = 20; 
power_prop_coeff     = 4.7;
power_indep_coeff    = 130;
noise_density        = float(10)^(-20.4);
RB_bandwidth         = 180e3;
eta = 2;

In [2]:
using JuMP

In [3]:
pathloss_matrix=rand(total_nb_users,nb_sectors,nb_RBs);

In [4]:
m = Model()
@defVar(m, PowerAllocationMatrix[1:nb_sectors,1:nb_RBs] >= log(min_power_per_RB))

PowerAllocationMatrix[i,j] ≥ -2.3025850929940455 for all i in {1,2..8,9}, j in {1,2..14,15}

In [5]:
@setNLObjective(m, Max, sum{log((exp(PowerAllocationMatrix[j,k])*pathloss_matrix[i,j,k])/(RB_bandwidth*noise_density + sum{exp(PowerAllocationMatrix[j_p,k])*pathloss_matrix[i,j_p,k], j_p=1:nb_sectors; j_p != j})), i=1:nb_users_per_sector, j=1:nb_sectors, k=1:nb_RBs} - eta *(power_prop_coeff*sum{exp(PowerAllocationMatrix[j,k]), j=1:nb_sectors, k=1:nb_RBs}+nb_sectors*power_indep_coeff))

In [6]:
for j=1:nb_sectors
    for k=1:nb_RBs
        @addConstraint(m, PowerAllocationMatrix[j,k] >= log(min_power_per_RB))
    end
end
for j=1:nb_sectors
    @addNLConstraint(m, sum{exp(PowerAllocationMatrix[j,k]), k=1:nb_RBs} <=max_power_per_sector)
end

In [7]:
status = solve(m)


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************

This is Ipopt version 3.12.1, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:        0
Number of nonzeros in inequality constraint Jacobian.:      270
Number of nonzeros in Lagrangian Hessian.............:      810

Total number of variables............................:      135
                     variables with only lower bounds:      135
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equa

:Optimal

In [12]:
log_power_sol=getValue(PowerAllocationMatrix);

In [19]:
power_sol=zeros(nb_sectors,nb_RBs);
for j=1:nb_sectors
    for k=1:nb_RBs
        power_sol[j,k] = exp(log_power_sol[j,k]);
    end
end

In [25]:
sum(power_sol,2)

9x1 Array{Float64,2}:
 1.55349
 1.56777
 1.59409
 1.57238
 1.60501
 1.58142
 1.53189
 1.58875
 1.63813